In [ ]:
#tfrrs scraper

In [1]:
%pip install -q google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 27.7 MB/s eta 0:00:00


In [2]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget https://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,246 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,759 kB]
Get:13 http://security.ubuntu.com/ubun

In [3]:
!pip install selenium webdriver-manager

import sys
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import tempfile

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Install and set up ChromeDriver
# This will automatically download the correct version for the Chrome browser in Colab
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

print("WebDriver initialized successfully with matching Chrome version.")

WebDriver initialized successfully with matching Chrome version.


In [4]:
import google_colab_selenium as gs
from io import BytesIO
from PIL import Image
import html5lib
import lxml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.image as mpimg
import datetime as dt
import os
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
driver.quit()

In [6]:
def get_element_text(parent_element, by, value, default="N/A"):
    try:
        return parent_element.find_element(by, value).text.strip()
    except NoSuchElementException:
        return default

def get_link_data(parent_element, by, value, default_text="N/A", default_href="#"):
    try:
        link = parent_element.find_element(by, value).find_element(By.TAG_NAME, "a")
        return link.text.strip(), link.get_attribute('href')
    except NoSuchElementException:
        try:
            text_content = parent_element.find_element(by, value).text.strip()
            return text_content, default_href
        except NoSuchElementException:
            return default_text, default_href

In [ ]:
# tffrs testing

url = 'https://tf.tfrrs.org/archives.html'

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
driver.get(url)
time.sleep(2)

links = driver.find_element(By.LINK_TEXT, '2023 OUTDOOR')
#links.click()
time.sleep(2)

#acc = driver.find_element(By.LINK_TEXT, 'ACC')
target_link_xpath = "//a[contains(@href, 'ACC') and @data-turbo-frame='_top']"
wait = WebDriverWait(driver, 10)
acc_link = wait.until(EC.presence_of_element_located((By.XPATH, target_link_xpath)))
print(acc_link.get_attribute('href'))
# Use JavaScript to click
driver.execute_script("arguments[0].click();", acc_link)
print("Clicked using JavaScript.")

print(driver.title)

https://tf.tfrrs.org/lists/4756/_ACC_Outdoor_Performance_List
Clicked using JavaScript.
TFRRS | ACC Outdoor Performance List


In [ ]:
#starting from acc link

acc_url = "https://www.tfrrs.org/lists/4756/_ACC_Outdoor_Performance_List"
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
driver.get(acc_url)
time.sleep(2)
print(driver.title)
# change filter to 500 runners
search = driver.find_element(By.CSS_SELECTOR, "select.search")
driver.execute_script("arguments[0].click();", search)
time.sleep(1)
athlete_options = search.find_elements(By.CSS_SELECTOR, "option")
athlete_options[8].click()
time.sleep(3)


In [11]:
def get_athlete_data(driver, all_row_data):
    print(driver.title)
    wait = WebDriverWait(driver, 20)
    row_css_selector = "div.performance-list-row"
    try:
        original_first_row = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, row_css_selector)))
        # change filter to 5k
        select_element_event = wait.until(EC.element_to_be_clickable((By.ID, "event")))
        select_object_event = Select(select_element_event)
        select_object_event.select_by_value("21")
        time.sleep(1)

        # change filter to 500 runners
        select_element_search = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "select.search")))
        select_object_search = Select(select_element_search)
        select_object_search.select_by_index(8)
        time.sleep(1)

        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, row_css_selector)))
        row_elements = driver.find_elements(By.CSS_SELECTOR, row_css_selector)

        print(f"Found {len(row_elements)} rows.")

        js_get_row_data = """
            const row = arguments[0];
            const data = {};

            const get_text = (selector) => {
                const el = row.querySelector(selector);
                return el ? el.textContent.trim() : "N/A";
            };

            const get_link = (selector) => {
                const el = row.querySelector(selector + ' a');
                if (el) {
                    return { text: el.textContent.trim(), href: el.href };
                }
                // Fallback if no link
                const parent_el = row.querySelector(selector);
                return { text: parent_el ? parent_el.textContent.trim() : "N/A", href: "#" };
            };

            data.rank = get_text("div.col-place");

            const athlete = get_link("div.col-athlete");
            data.athlete_name = athlete.text;
            data.athlete_link = athlete.href;

            data.year = get_text("div.col-narrow[data-label='Year']");

            const team = get_link("div.col-team");
            data.team_name = team.text;
            data.team_link = team.href;

            data.time = get_text("div.col-narrow[data-label='Time']");
            data.meet_date = get_text("div.col-narrow[data-label='Meet Date']");

            return data;
            """
        for row_element in row_elements:
            current_row_data = driver.execute_script(js_get_row_data, row_element)
            all_row_data.append(current_row_data)

    except TimeoutException:
        print(f"  Timed out waiting for elements on page: {driver.title}. Skipping this URL.")
    except Exception as e:
        print(f"  An unexpected error occurred on {driver.title}: {e}")
        import traceback
        traceback.print_exc()


In [ ]:
# looping over years
url = 'https://tf.tfrrs.org/archives.html'

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
driver.get(url)
time.sleep(2)

years = driver.find_element(By.CSS_SELECTOR, "ul.nav")
indiv_years = years.find_elements(By.CSS_SELECTOR, "li")
year_links = []
for year in indiv_years:
    print(year.text)
    year_links.append(year.find_element(By.TAG_NAME, "a").get_attribute('href'))
print(year_links)

# years also have individual pages -- https://tf.tfrrs.org/college_archives_tab.html?outdoor=0&year=2024

2024 OUTDOOR
2024 INDOOR
2023 OUTDOOR
2023 INDOOR
2022 OUTDOOR
2022 INDOOR
2021 OUTDOOR
2021 INDOOR
2020 OUTDOOR
2020 INDOOR
2019 OUTDOOR
2019 INDOOR
2018 OUTDOOR
2018 INDOOR
2017 OUTDOOR
2017 INDOOR
2016 OUTDOOR
2016 INDOOR
2015 OUTDOOR
2015 INDOOR
2014 OUTDOOR
2014 INDOOR
2013 OUTDOOR
2013 INDOOR
2012 OUTDOOR
2012 INDOOR
2011 OUTDOOR
2011 INDOOR
2010 OUTDOOR
2010 INDOOR
['https://tf.tfrrs.org/archives.html#a2024outdoor', 'https://tf.tfrrs.org/archives.html#a2024indoor', 'https://tf.tfrrs.org/archives.html#a2023outdoor', 'https://tf.tfrrs.org/archives.html#a2023indoor', 'https://tf.tfrrs.org/archives.html#a2022outdoor', 'https://tf.tfrrs.org/archives.html#a2022indoor', 'https://tf.tfrrs.org/archives.html#a2021outdoor', 'https://tf.tfrrs.org/archives.html#a2021indoor', 'https://tf.tfrrs.org/archives.html#a2020outdoor', 'https://tf.tfrrs.org/archives.html#a2020indoor', 'https://tf.tfrrs.org/archives.html#a2019outdoor', 'https://tf.tfrrs.org/archives.html#a2019indoor', 'https://tf.tfrrs.

In [8]:
# getting all conference links
url = 'https://tf.tfrrs.org/archives.html'

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
driver.get(url)
time.sleep(2)

conference_link_elements = driver.find_elements(By.XPATH, "//a[contains(@href, 'Performance') and contains(@href, 'gender=m') and not(contains(@href, 'Region')) and not(contains(@href, 'Indoor')) and @data-turbo-frame='_top']")
print(len(conference_link_elements))
conference_link_elements = conference_link_elements[::2]
conference_links = []

for link in conference_link_elements:
    print(link.get_attribute('href'))
    conference_links.append(link.get_attribute('href'))
print(len(conference_links))

2804
https://tf.tfrrs.org/lists/4756/_ACC_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4750/ASUN_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4755/America_East_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4754/Atlantic_10_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4782/BIG_EAST_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4752/Big_12_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4757/Big_Sky_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4758/Big_South_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4759/Big_Ten_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4760/Big_West_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4762/CAA_Colonial_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4761/Conference_USA_Outdoor_Performance_List?gender=m
https://tf.tfrrs.org/lists/4763/Horizon_League_Outdoor_Performance_List?gender=m
https://

Couple of options going forward. I should probably loop over the years archives and make a new dict for each one that then loops over each of the conferences in that year, making a dict for each one with the athletes in it. The other option is to loop over all of them at once which eliminates the need to scrape as much, but I think it will ultimately be easier to keep track of years and conferences this way. (Update: each result already has the year stored within the date, and conferences really do not matter for what I am doing, so I am just going to loop over all 1300 conferences)

In [17]:
all_results = []

In [21]:
for result in all_results:
    print(result)

Output hidden; open in https://colab.research.google.com to view.

In [31]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
i = 1359
for conf in conference_links[1359:]:
    print(i)
    i += 1
    driver.get(conf)

    get_athlete_data(driver, all_results)

print(len(all_results))

1359
TFRRS | America East Outdoor Performance List
Found 62 rows.
1360
TFRRS | Big Sky Outdoor Performance List
Found 42 rows.
1361
TFRRS | Big South Outdoor Performance List
Found 36 rows.
1362
TFRRS | Big Ten Outdoor Performance List
Found 66 rows.
1363
TFRRS | Big West Conference Outdoor Performance List
Found 63 rows.
1364
TFRRS | CAA Outdoor Performance List
Found 52 rows.
1365
TFRRS | Horizon League Outdoor Performance List
Found 49 rows.
1366
TFRRS | Ivy League Outdoor Performance List
Found 100 rows.
1367
TFRRS | MAAC Outdoor Performance List
Found 50 rows.
1368
TFRRS | Mets Outdoor Performance List
Found 93 rows.
1369
TFRRS | Missouri Valley Outdoor Performance List
Found 57 rows.
1370
TFRRS | Mountain West Outdoor Performance List
Found 37 rows.
1371
TFRRS | Northeast Conference Outdoor Performance List
Found 20 rows.
  An unexpected error occurred on TFRRS | Northeast Conference Outdoor Performance List: Message: stale element reference: stale element not found in the curren

Traceback (most recent call last):
  File "<ipython-input-11-be41c168762c>", line 61, in get_athlete_data
    current_row_data = driver.execute_script(js_get_row_data, row_element)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/selenium/webdriver/remote/webdriver.py", line 544, in execute_script
    return self.execute(command, {"script": script, "args": converted_args})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/selenium/webdriver/remote/webdriver.py", line 447, in execute
    self.error_handler.check_response(response)
  File "/usr/local/lib/python3.11/dist-packages/selenium/webdriver/remote/errorhandler.py", line 232, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.StaleElementReferenceException: Message: stale element reference: stale element not found in the current

TFRRS | NEICAAA Outdoor Performance List
Found 200 rows.
1373
TFRRS | Ohio Valley Conference Outdoor Performance List
Found 40 rows.
1374
TFRRS | Pac-10 Performance List
Found 72 rows.
1375
TFRRS | Patriot League Outdoor Performance List
Found 75 rows.
1376
TFRRS | Sun Belt Outdoor Performance List
Found 56 rows.
1377
TFRRS | WAC Outdoor Performance List
Found 37 rows.
1378
TFRRS | CCAA Outdoor Performance List
Found 47 rows.
1379
TFRRS | Conference Carolinas Outdoor Performance List
Found 27 rows.
1380
TFRRS | GLIAC Outdoor Performance List
Found 83 rows.
1381
TFRRS | GNAC Outdoor Performance List
Found 30 rows.
1382
TFRRS | Lone Star Conference Outdoor Performance List
Found 40 rows.
1383
TFRRS | Mets Outdoor Performance List
Found 93 rows.
1384
TFRRS | NEICAAA Outdoor Performance List
Found 200 rows.
1385
TFRRS | Northeast-10 Outdoor Performance List
Found 59 rows.
1386
TFRRS | Northern Sun Outdoor Performance List
Found 47 rows.
1387
TFRRS | PSAC Outdoor Performance List
Found 50 r

Traceback (most recent call last):
  File "<ipython-input-11-be41c168762c>", line 10, in get_athlete_data
    select_object_event.select_by_value("21")
  File "/usr/local/lib/python3.11/dist-packages/selenium/webdriver/support/select.py", line 83, in select_by_value
    raise NoSuchElementException(f"Cannot locate option with value: {value}")
selenium.common.exceptions.NoSuchElementException: Message: Cannot locate option with value: 21; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception



TFRRS | Patriot League Performance List
  An unexpected error occurred on TFRRS | Patriot League Performance List: Message: Cannot locate option with value: 21; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception

80859


Traceback (most recent call last):
  File "<ipython-input-11-be41c168762c>", line 10, in get_athlete_data
    select_object_event.select_by_value("21")
  File "/usr/local/lib/python3.11/dist-packages/selenium/webdriver/support/select.py", line 83, in select_by_value
    raise NoSuchElementException(f"Cannot locate option with value: {value}")
selenium.common.exceptions.NoSuchElementException: Message: Cannot locate option with value: 21; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception



In [33]:
df = pd.DataFrame(all_results)
df.to_csv('/content/drive/MyDrive/tfrrs_dataall.csv')

In [37]:
len(all_results)
df.tail()

,athlete_link,athlete_name,meet_date,rank,team_link,team_name,time,year
80854,https://tf.tfrrs.org/athletes/2857659/Mt_Hood_...,"Hughes, Wes","Mar 5, 2010",29,https://tf.tfrrs.org/teams/tf/OR_jcollege_m_Mt...,Mt. Hood CC,17:39.30,SO
80855,https://tf.tfrrs.org/athletes/2666531/Everett_...,"Baker, Alex","Apr 9, 2010",30,https://tf.tfrrs.org/teams/tf/WA_jcollege_m_Ev...,Everett CC,17:50.01,FR
80856,https://tf.tfrrs.org/athletes/2666573/Everett_...,"Trowbridge, Seth","Mar 27, 2010",31,https://tf.tfrrs.org/teams/tf/WA_jcollege_m_Ev...,Everett CC,17:50.20,SO
80857,https://tf.tfrrs.org/athletes/2306116/Lane_CC/...,"Bateham, Erik","May 15, 2010",32,https://tf.tfrrs.org/teams/tf/OR_jcollege_m_La...,Lane CC,17:56.06,FR
80858,https://tf.tfrrs.org/athletes/2722293/SW_Orego...,"Ramirez, Sam","Mar 6, 2010",33,https://tf.tfrrs.org/teams/tf/OR_jcollege_m_SW...,SW Oregon CC,18:19.41,FR


In [ ]:
results = []
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
driver.get(conference_links[0])
time.sleep(2)
get_athlete_data(driver, results)
print(results)

Found 105 rows.
Wolfe, Parker
Mullarkey, David
Martin, Gary
Mountain, Nathan
Phillip, Alex
Anthony, Will
Strand, Ethan
Gebhardt, Jake
Hansen, Rocky
Porter, Wes
Solomon, Carter
Sands, Colton
Kibiwot, Ian
Coleman, Ethan
Singleton, CJ
Berg, Tyler
Wachtel, Justin
Ackley, Daelen
Cloud, Zachary
Coogan, Will
Gardner, Brett
Miller, Jack
Anderson, Patrick
Ackley, Connor
Vorbach, David
Jones, Andrew
Row 1: {'place': '1', 'athlete_name': 'Wolfe, Parker', 'athlete_link': 'https://tf.tfrrs.org/athletes/7908131/North_Carolina/Parker_Wolfe', 'year': 'JR-3', 'team_name': 'North Carolina', 'team_link': 'https://tf.tfrrs.org/teams/tf/NC_college_m_North_Carolina.html', 'time': '13:19.38', 'time_results_link': 'https://tf.tfrrs.org/results/86920/5278891/ACC_Outdoor_Track__Field_Championships/5000-Meters', 'meet_name': 'ACC Outdoor Track & Field Championships', 'meet_link': 'https://tf.tfrrs.org/results/86920/ACC_Outdoor_Track__Field_Championships'}
Row 2: {'place': '2', 'athlete_name': 'Mullarkey, David',

In [ ]:
def get_element_text(parent_element, by, value, default="N/A"):
    return parent_element.find_element(by, value).text.strip()

def get_link_data(parent_element, by, value, default_text="N/A", default_href="#"):
    link = parent_element.find_element(by, value).find_element(By.TAG_NAME, "a")
    return link.text.strip(), link.get_attribute('href')
def get_athlete_data(driver, all_row_data):
    print(driver.title)
    # change filter to 500 runners
    search = driver.find_element(By.CSS_SELECTOR, "select.search")
    driver.execute_script("arguments[0].click();", search)
    time.sleep(1)
    athlete_options = search.find_elements(By.CSS_SELECTOR, "option")
    athlete_options[8].click()
    time.sleep(3)

    # change filter to 5k
    event_filter = driver.find_element(By.ID, "event")
    driver.execute_script("arguments[0].click();", event_filter)
    time.sleep(1)
    event_options = event_filter.find_elements(By.CSS_SELECTOR, "option")
    event_options[6].click()
    time.sleep(3)

    row_css_selector = "div.performance-list-row"

    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, row_css_selector)))
    row_elements = driver.find_elements(By.CSS_SELECTOR, row_css_selector)


    print(f"Found {len(row_elements)} rows.")
    # loop over all athlete rows and get their data
    for index, row_element in enumerate(row_elements):
        current_row_data = {}

        current_row_data['place'] = get_element_text(row_element, By.CSS_SELECTOR, "div.col-place")
        athlete_name, athlete_link = get_link_data(row_element, By.CSS_SELECTOR, "div.col-athlete")
        current_row_data['athlete_name'] = athlete_name
        current_row_data['athlete_link'] = athlete_link
        current_row_data['year'] = get_element_text(row_element, By.CSS_SELECTOR, "div.col-narrow[data-label='Year']")
        team_name, team_href = get_link_data(row_element, By.CSS_SELECTOR, "div.col-team")
        current_row_data['team_name'] = team_name
        current_row_data['team_link'] = team_href

        time_value = get_element_text(row_element, By.CSS_SELECTOR, "div.col-narrow[data-label='Time']")
        current_row_data['time'] = time_value
        js_command = "return arguments[0].querySelector(\"div[data-label='Meet Date']\").textContent;"

        # Execute the script with the row_element as the context
        meet_date = driver.execute_script(js_command, row_element)
        current_row_data['meet_date'] = meet_date.strip() if meet_date else "N/A"

        all_row_data.append(current_row_data)

all_results = []
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

for conf in conference_links[:1]:
    driver.get(conf)
    time.sleep(1)
    get_athlete_data(driver, all_results)
    time.sleep(1)